# Training Models in AI Platform Training

## 1. Local Python Runner

In [29]:
import os
import numpy as np
PROJECT = 'crazy-hippo-01' 
BUCKET = 'crazy-hippo-01' 
REGION = 'us-central1' 
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'

In [3]:
%%bash
python3 -W ignore -m trainer.task \
    --x_train_data_paths='gs://crazy-hippo-01/dataset/x_train.csv' \
    --y_train_data_paths='gs://crazy-hippo-01/dataset/y_train.csv' \
    --x_eval_data_paths='gs://crazy-hippo-01/dataset/x_val.csv'  \
    --y_eval_data_paths='gs://crazy-hippo-01/dataset/y_val.csv'  \
    --epochs=5 \
    --train_batch_size=32 \
    --output_dir=gs://$BUCKET/ai-platform/ml_engine_training \
    --num_eval_examples=100


Epoch 1/5
122/122 [==============================] - 2s 16ms/step - loss: 0.5529 - acc: 0.7377 - val_loss: 0.4442 - val_acc: 0.7709
Epoch 2/5
122/122 [==============================] - 2s 13ms/step - loss: 0.4210 - acc: 0.8061 - val_loss: 0.3808 - val_acc: 0.8197
Epoch 3/5
122/122 [==============================] - 1s 12ms/step - loss: 0.3795 - acc: 0.8258 - val_loss: 0.3707 - val_acc: 0.8231
Epoch 4/5
122/122 [==============================] - 1s 12ms/step - loss: 0.3637 - acc: 0.8253 - val_loss: 0.3643 - val_acc: 0.8200
Epoch 5/5
20/20 [==============================] - 0s 7ms/step - loss: 0.3594 - acc: 0.8375
Test Loss: 0.35938018560409546 Test Accuracy: 0.8374999761581421
Model Saved in gs://crazy-hippo-01/ai-platform/ml_engine_training/export/savedmodel
0.8374999761581421 0.35938018560409546


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


## 2. Run local using GCLOUD

In [4]:
! gcloud config set ml_engine/local_python $(which python3)

Updated property [ml_engine/local_python].


In [5]:
%%bash
gcloud ai-platform local train \
    --module-name=trainer.task \
    --package-path=${PWD}/trainer \
    -- \
    --x_train_data_paths="gs://crazy-hippo-01/dataset/x_train.csv" \
    --y_train_data_paths="gs://crazy-hippo-01/dataset/y_train.csv" \
    --x_eval_data_paths="gs://crazy-hippo-01/dataset/x_val.csv"  \
    --y_eval_data_paths="gs://crazy-hippo-01/dataset/y_val.csv"  \
    --epochs=5 \
    --train_batch_size=32 \
    --output_dir=gs://$BUCKET/ai-platform/ml_engine_training \
    --num_eval_examples=100 

Epoch 1/5
122/122 [==============================] - 2s 19ms/step - loss: 0.5587 - acc: 0.7090 - val_loss: 0.4372 - val_acc: 0.7641
Epoch 2/5
122/122 [==============================] - 1s 12ms/step - loss: 0.4244 - acc: 0.7590 - val_loss: 0.4037 - val_acc: 0.7644
Epoch 3/5
122/122 [==============================] - 2s 15ms/step - loss: 0.4010 - acc: 0.7774 - val_loss: 0.3888 - val_acc: 0.8191
Epoch 4/5
122/122 [==============================] - 1s 12ms/step - loss: 0.3781 - acc: 0.8268 - val_loss: 0.3763 - val_acc: 0.8156
Epoch 5/5
20/20 [==============================] - 0s 6ms/step - loss: 0.3551 - acc: 0.8328
Test Loss: 0.3550998568534851 Test Accuracy: 0.832812488079071
Model Saved in gs://crazy-hippo-01/ai-platform/ml_engine_training/export/savedmodel


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


## 3. Submit job to AI Platform Training

In [30]:
%%bash
OUTDIR=gs://${BUCKET}/ai-platform/ml_engine_training/earnings
JOBNAME=earnings_$(date -u +%y%m%d_%H%M%S)

gcloud ai-platform jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=trainer.task \
    --package-path=${PWD}/trainer \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://$BUCKET \
    --scale-tier=BASIC \
    --runtime-version=$TFVERSION \
    --python-version=3.7 \
    -- \
    --x_train_data_paths="gs://crazy-hippo-01/dataset/x_train.csv" \
    --y_train_data_paths="gs://crazy-hippo-01/dataset/y_train.csv" \
    --x_eval_data_paths="gs://crazy-hippo-01/dataset/x_val.csv"  \
    --y_eval_data_paths="gs://crazy-hippo-01/dataset/y_val.csv"  \
    --epochs=5 \
    --train_batch_size=32 \
    --output_dir=$OUTDIR \
    --num_eval_examples=100 

jobId: earnings_210207_135733
state: QUEUED


Job [earnings_210207_135733] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe earnings_210207_135733

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs earnings_210207_135733
